# import

In [ ]:
import random
import pandas as pd
import numpy as np
import os

from sklearn.preprocessing import LabelEncoder
from sklearn.feature_extraction.text import CountVectorizer ,TfidfVectorizer

import torch

from tqdm.auto import tqdm
from sklearn.metrics import f1_score

In [ ]:
device = torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu')
device

device(type='cuda')

# mount

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
DATA_PATH = "/content/drive/MyDrive/Dacon/Data/발화자의 감정인식 AI/"
SEED = 42

In [ ]:
if not os.path.isdir("image"):
    !cp "{DATA_PATH}open.zip" "open.zip"
    !unzip -qq "open.zip"

cp: cannot stat '/content/drive/MyDrive/Dacon/Data/발화자의 감정인식 AI/open.zip': No such file or directory
unzip:  cannot find or open open.zip, open.zip.zip or open.zip.ZIP.


In [ ]:
def seed_everything(seed):
    random.seed(seed)
    os.environ['PYTHONHASHSEED'] = str(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = True

In [ ]:
train = pd.read_csv(f'{DATA_PATH}train.csv')
test = pd.read_csv(f'{DATA_PATH}test.csv')
info = pd.read_csv(f'{DATA_PATH}data_info.csv',encoding='euc-kr')
info

,feature,information,type
0,ID,고유번호,object
1,Utterance,발화문,object
2,Speaker,발화자,object
3,Dialogue_ID,Dialogue 구분 번호,int
4,Target,감정,object


In [ ]:
train

,ID,Utterance,Speaker,Dialogue_ID,Target
0,TRAIN_0000,also I was the point person on my company’s tr...,Chandler,0,neutral
1,TRAIN_0001,You must’ve had your hands full.,The Interviewer,0,neutral
2,TRAIN_0002,That I did. That I did.,Chandler,0,neutral
3,TRAIN_0003,So let’s talk a little bit about your duties.,The Interviewer,0,neutral
4,TRAIN_0004,My duties? All right.,Chandler,0,surprise
...,...,...,...,...,...
9984,TRAIN_9984,You or me?,Chandler,1038,neutral
9985,TRAIN_9985,"I got it. Uh, Joey, women don't have Adam's ap...",Ross,1038,neutral
9986,TRAIN_9986,"You guys are messing with me, right?",Joey,1038,surprise
9987,TRAIN_9987,Yeah.,All,1038,neutral


In [ ]:
test

,ID,Utterance,Speaker,Dialogue_ID
0,TEST_0000,Why do all the coffee cups have figures below?,Mark,0
1,TEST_0001,"Oh. It's so Monica can follow. Of this way, if...",Rachell,0
2,TEST_0002,You know what?,Rachell,0
3,TEST_0003,"Come on, Lydia, you can do it.",Joeyy,1
4,TEST_0004,To push!,Joeyy,1
...,...,...,...,...
2605,TEST_2605,"Yeah, I mean, go Ross, no one will even notice...",Rachell,279
2606,TEST_2606,They don't listen to me?,Rossi,279
2607,TEST_2607,"Of course, they listen to you! Everyone listen...",Rachell,279
2608,TEST_2608,"Monica, do you really think I should try this ...",Rossi,279


In [ ]:
train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 9989 entries, 0 to 9988
Data columns (total 5 columns):
 #   Column       Non-Null Count  Dtype 
---  ------       --------------  ----- 
 0   ID           9989 non-null   object
 1   Utterance    9989 non-null   object
 2   Speaker      9989 non-null   object
 3   Dialogue_ID  9989 non-null   int64 
 4   Target       9989 non-null   object
dtypes: int64(1), object(4)
memory usage: 390.3+ KB


In [ ]:
test.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2610 entries, 0 to 2609
Data columns (total 4 columns):
 #   Column       Non-Null Count  Dtype 
---  ------       --------------  ----- 
 0   ID           2610 non-null   object
 1   Utterance    2610 non-null   object
 2   Speaker      2610 non-null   object
 3   Dialogue_ID  2610 non-null   int64 
dtypes: int64(1), object(3)
memory usage: 81.7+ KB


In [ ]:
train.nunique()

ID             9989
Utterance      8931
Speaker         260
Dialogue_ID    1038
Target            7
dtype: int64

In [ ]:
test.nunique()

ID             2610
Utterance      2371
Speaker         100
Dialogue_ID     280
dtype: int64

In [ ]:
le = LabelEncoder()
train["Target"] = le.fit_transform(train["Target"].to_numpy())
train

,ID,Utterance,Speaker,Dialogue_ID,Target
0,TRAIN_0000,also I was the point person on my company’s tr...,Chandler,0,4
1,TRAIN_0001,You must’ve had your hands full.,The Interviewer,0,4
2,TRAIN_0002,That I did. That I did.,Chandler,0,4
3,TRAIN_0003,So let’s talk a little bit about your duties.,The Interviewer,0,4
4,TRAIN_0004,My duties? All right.,Chandler,0,6
...,...,...,...,...,...
9984,TRAIN_9984,You or me?,Chandler,1038,4
9985,TRAIN_9985,"I got it. Uh, Joey, women don't have Adam's ap...",Ross,1038,4
9986,TRAIN_9986,"You guys are messing with me, right?",Joey,1038,6
9987,TRAIN_9987,Yeah.,All,1038,4


In [ ]:
!pip install kiwipiepy

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 1.6 MB 5.0 MB/s 
     |████████████████████████████████| 30.5 MB 46.7 MB/s 
  Created wheel for kiwipiepy-model: filename=kiwipiepy_model-0.14.0-py3-none-any.whl size=30560039 sha256=5588a6cd647e2050fceef0152b4f4ae2b2b9404ed3cf13e490874e4c2e389e3c
  Stored in directory: /root/.cache/pip/wheels/97/36/50/1fc99e4a3ce636ed30739cc2cb333b36e6e3778f365f1adf77
Successfully built kiwipiepy-model


In [ ]:
from kiwipiepy import Kiwi

kiwi = Kiwi()
kiwi.prepare()

In [ ]:
lst = train["Utterance"].tolist()
gen = kiwi.analyze(lst)
train_overview = []

for tokens in tqdm(gen,total=len(lst)):
    token = [ token.form for token in tokens[0][0] if token.tag[0] in "SF"or"SL" ]
    train_overview.append(" ".join(token))

lst = test["Utterance"].tolist()
gen = kiwi.analyze(lst)
test_overview = []
for tokens in tqdm(gen,total=len(lst)):
    token = [ token.form for token in tokens[0][0] if token.tag[0] in "SF"or"SL" ]
    test_overview.append(" ".join(token))

  0%|          | 0/9989 [00:00<?, ?it/s]

  0%|          | 0/2610 [00:00<?, ?it/s]

# 전처리된 데이터 csv로 저장하고 다시 불러오기(메모리를 효율적으로 사용)

In [ ]:
# def csv_to_parquet(csv_path, save_name):
#     df = pd.read_csv(csv_path)
#     df.to_parquet(f'./{save_name}.parquet')
#     del df
#     gc.collect()
#     print(save_name, 'Done.')

In [ ]:
# csv_to_parquet('/content/drive/MyDrive/Dacon/Data/train.csv', 'train')
# csv_to_parquet('/content/drive/MyDrive/Dacon/Data/test.csv', 'test')

In [ ]:
# train = pd.read_parquet('/content/train.parquet')
# test = pd.read_parquet('/content/test.parquet')

In [ ]:
!pip install transformers
from transformers import pipeline

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 5.5 MB 5.1 MB/s 
     |████████████████████████████████| 163 kB 68.3 MB/s 
     |████████████████████████████████| 7.6 MB 53.1 MB/s 


In [ ]:
model_name = "roberta-base"

In [ ]:
from transformers import RobertaTokenizer, RobertaModel
tokenizer = RobertaTokenizer.from_pretrained('roberta-base')
model = RobertaModel.from_pretrained('roberta-base')
text = "Replace me by any text you'd like."
encoded_input = tokenizer(text, return_tensors='pt')
output = model(**encoded_input)

Downloading:   0%|          | 0.00/899k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/456k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/481 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/501M [00:00<?, ?B/s]

Some weights of the model checkpoint at roberta-base were not used when initializing RobertaModel: ['lm_head.layer_norm.weight', 'lm_head.dense.weight', 'lm_head.layer_norm.bias', 'lm_head.decoder.weight', 'lm_head.dense.bias', 'lm_head.bias']
- This IS expected if you are initializing RobertaModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing RobertaModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [ ]:
# from transformers import TFAutoModelForMaskedLM, AutoTokenizer

# tokenizer = AutoTokenizer.from_pretrained(model_name)
# model = TFAutoModelForMaskedLM.from_pretrained(model_name)

In [ ]:
train_overview = np.array(train_overview)
test_overview = np.array(test_overview)
target = train.Target.to_numpy()

In [ ]:
train_overview

array(['also I was the point person on my company ’ s transition from the KL - 5 to GR - 6 system .',
       'You must ’ ve had your hands full .', 'That I did . That I did .',
       ..., 'You guys are messing with me , right ?', 'Yeah .',
       'That was a good one . For a second there , I was like , " whoa . "'],
      dtype='<U351')

In [ ]:
token = tokenizer(train_overview[0], add_special_tokens=True,padding="max_length", truncation=True)
token

{'input_ids': [0, 19726, 38, 21, 5, 477, 621, 15, 127, 138, 44, 27, 579, 3868, 31, 5, 26544, 111, 195, 7, 8837, 111, 231, 467, 479, 2, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1

In [ ]:
class MyDataset(torch.utils.data.Dataset):
    def __init__(self ,tokenizer , x, y = None ): 
        self.tokenizer = tokenizer
        self.x = x
        self.y = y
    def __len__(self): 
        return self.x.shape[0]
    def __getitem__(self, idx): 
        item = {}
        item["x"] = self.__tokenizer(self.x[idx]) 
        if self.y is not None:
            item["y"] = torch.tensor(self.y[idx])
        return item
    def __tokenizer(self,text):
        inputs = self.tokenizer(text, add_special_tokens=True,padding="max_length", truncation=True)
        for k, v in inputs.items(): 
            inputs[k] = torch.LongTensor(v) 
        return inputs

In [ ]:
dt = MyDataset(tokenizer,train_overview,target)
dl = torch.utils.data.DataLoader(dt, batch_size=8,shuffle=False) 
batch = next(iter(dl))
batch

{'x': {'input_ids': tensor([[    0, 19726,    38,  ...,     1,     1,     1],
         [    0,  1185,   531,  ...,     1,     1,     1],
         [    0,  1711,    38,  ...,     1,     1,     1],
         ...,
         [    0,  5975,    47,  ...,     1,     1,     1],
         [    0,   100,   192,  ...,     1,     1,     1],
         [    0,  1708,    89,  ...,     1,     1,     1]]), 'attention_mask': tensor([[1, 1, 1,  ..., 0, 0, 0],
         [1, 1, 1,  ..., 0, 0, 0],
         [1, 1, 1,  ..., 0, 0, 0],
         ...,
         [1, 1, 1,  ..., 0, 0, 0],
         [1, 1, 1,  ..., 0, 0, 0],
         [1, 1, 1,  ..., 0, 0, 0]])}, 'y': tensor([4, 4, 4, 4, 6, 4, 4, 4])}

In [ ]:
model = RobertaModel.from_pretrained(model_name)

Some weights of the model checkpoint at roberta-base were not used when initializing RobertaModel: ['lm_head.layer_norm.weight', 'lm_head.dense.weight', 'lm_head.layer_norm.bias', 'lm_head.decoder.weight', 'lm_head.dense.bias', 'lm_head.bias']
- This IS expected if you are initializing RobertaModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing RobertaModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [ ]:
batch["x"].keys()

dict_keys(['input_ids', 'attention_mask'])

In [ ]:
outputs = model(**batch["x"])
outputs.keys()

odict_keys(['last_hidden_state', 'pooler_output'])

In [ ]:
model.config.hidden_size

768

In [ ]:
class Net(torch.nn.Module):
    def __init__(self, model_name): 
        super().__init__()
        self.model = RobertaModel.from_pretrained(model_name)
        self.output_layer = torch.nn.Linear(self.model.config.hidden_size, 128)
    def forward(self, x):
        x = self.model(**x) 
        x = self.output_layer(x[1]) # globar pooling
        return x

In [ ]:
train.Target.nunique()

7

In [ ]:
def train_loop(dataloader,model,loss_fn,optimizer,device):
    epoch_loss = 0 
    model.train() 
    for batch in tqdm(dataloader): 
        pred = model(batch["x"].to(device)) 
        loss = loss_fn(pred, batch["y"].to(device)) 
        
        optimizer.zero_grad() 
        loss.backward()  
        optimizer.step() 
        
        epoch_loss += loss.item() 

    epoch_loss /= len(dataloader) 

    return epoch_loss 

In [ ]:
@torch.no_grad() 
def test_loop(dataloader,model,loss_fn,device): 
    epoch_loss = 0
    model.eval() 

    
    pred_list = []
    softmax = torch.nn.Softmax(dim=1) 

    for batch in tqdm(dataloader):
        
        pred = model(batch["x"].to(device))
        if batch.get("y") is not None: 
            loss = loss_fn(pred, batch["y"].to(device))
            epoch_loss += loss.item()
        
        pred = softmax(pred)
        pred = pred.to("cpu").numpy() 
        pred_list.append(pred)

    epoch_loss /= len(dataloader)

    pred = np.concatenate(pred_list) 
    return epoch_loss , pred 

In [ ]:
n_splits = 5
batch_size = 16
epochs = 100
loss_fn = torch.nn.CrossEntropyLoss() # 다중분류 손실객체

In [ ]:
from sklearn.metrics import f1_score
from sklearn.model_selection import KFold
cv = KFold(n_splits=n_splits,shuffle=True, random_state=SEED)

In [ ]:
is_holdout = False
seed_everything(SEED)
best_score_list = []
for i,(tri,vai) in enumerate(cv.split(train_overview)):
    if i != 3:
        continue
    model = Net(model_name).to(device)
    optimizer = torch.optim.Adam(model.parameters(),lr=0.00001)

    train_dt = MyDataset(tokenizer,train_overview[tri],target[tri])
    valid_dt = MyDataset(tokenizer,train_overview[vai],target[vai])
    train_dl = torch.utils.data.DataLoader(train_dt, batch_size=batch_size, shuffle=True)
    valid_dl = torch.utils.data.DataLoader(valid_dt, batch_size=batch_size,shuffle=False)

    best_score = 0
    patience = 0

    for epoch in range(epochs):
        
        train_loss = train_loop(train_dl, model, loss_fn,optimizer,device )
        valid_loss , pred = test_loop(valid_dl, model, loss_fn,device  )
        
        pred = np.argmax(pred, axis=1) 
        true = target[vai] 
        score = f1_score(true, pred , average="weighted")
        print(train_loss,valid_loss,score)
        patience += 1
        if best_score < score:
            patience = 0
            best_score = score
            torch.save(model.state_dict(),f"{DATA_PATH}model_{i}.pth")

        if patience == 2:
            break
    print(f"Fold ({i}), BEST F1: {best_score}")
    best_score_list.append(best_score)

    torch.cuda.empty_cache()

    if is_holdout:
        break

In [ ]:
np.mean(best_score_list)

In [ ]:
MODEL_PATH = '/content/drive/MyDrive/Dacon/Data/발화자의 감정인식 AI/roberta'

In [ ]:
test_dt = MyDataset(tokenizer,test_overview)
test_dl = torch.utils.data.DataLoader(test_dt, batch_size=batch_size, shuffle=False)
pred_list = []

for i in range(n_splits):
    
    model = Net(model_name).to(device)
    state_dict = torch.load(f'{MODEL_PATH}/model_{i}.pth') 
    model.load_state_dict(state_dict)
        
    _ , pred = test_loop(test_dl, model, loss_fn,device  )
    pred_list.append(pred)
    
    if is_holdout:
        break

Some weights of the model checkpoint at roberta-base were not used when initializing RobertaModel: ['lm_head.layer_norm.weight', 'lm_head.dense.weight', 'lm_head.layer_norm.bias', 'lm_head.decoder.weight', 'lm_head.dense.bias', 'lm_head.bias']
- This IS expected if you are initializing RobertaModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing RobertaModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


  0%|          | 0/164 [00:00<?, ?it/s]

Some weights of the model checkpoint at roberta-base were not used when initializing RobertaModel: ['lm_head.layer_norm.weight', 'lm_head.dense.weight', 'lm_head.layer_norm.bias', 'lm_head.decoder.weight', 'lm_head.dense.bias', 'lm_head.bias']
- This IS expected if you are initializing RobertaModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing RobertaModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


  0%|          | 0/164 [00:00<?, ?it/s]

Some weights of the model checkpoint at roberta-base were not used when initializing RobertaModel: ['lm_head.layer_norm.weight', 'lm_head.dense.weight', 'lm_head.layer_norm.bias', 'lm_head.decoder.weight', 'lm_head.dense.bias', 'lm_head.bias']
- This IS expected if you are initializing RobertaModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing RobertaModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


  0%|          | 0/164 [00:00<?, ?it/s]

Some weights of the model checkpoint at roberta-base were not used when initializing RobertaModel: ['lm_head.layer_norm.weight', 'lm_head.dense.weight', 'lm_head.layer_norm.bias', 'lm_head.decoder.weight', 'lm_head.dense.bias', 'lm_head.bias']
- This IS expected if you are initializing RobertaModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing RobertaModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


  0%|          | 0/164 [00:00<?, ?it/s]

Some weights of the model checkpoint at roberta-base were not used when initializing RobertaModel: ['lm_head.layer_norm.weight', 'lm_head.dense.weight', 'lm_head.layer_norm.bias', 'lm_head.decoder.weight', 'lm_head.dense.bias', 'lm_head.bias']
- This IS expected if you are initializing RobertaModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing RobertaModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


  0%|          | 0/164 [00:00<?, ?it/s]

In [ ]:
pred = np.mean(pred_list,axis=0) 
pred = np.argmax(pred, axis=1) 
pred

array([4, 4, 4, ..., 3, 0, 4])

In [ ]:
submit = pd.read_csv(f"{DATA_PATH}sample_submission.csv")
submit["Target"] = le.inverse_transform(pred)
submit

,ID,Target
0,TEST_0000,neutral
1,TEST_0001,neutral
2,TEST_0002,neutral
3,TEST_0003,neutral
4,TEST_0004,joy
...,...,...
2605,TEST_2605,neutral
2606,TEST_2606,surprise
2607,TEST_2607,joy
2608,TEST_2608,anger


In [ ]:
submit.to_csv(f'{DATA_PATH}roberta_base_ALLfold.csv', index=False)